In [138]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(100)

def prox_l1(y,alpha):
    ## \arg\min\|x\|_1+\frac{1}{2\alpha}\|x-y\|_2^2
    x = y.copy()
    x[y>alpha] = x[y>alpha]-alpha
    x[y<-alpha] = x[y<-alpha]+alpha
    x[abs(y)-alpha<=0] = 0
    return x
def project_nonnegtive(x):
    x[x<0]=0
    return x
def prox_l1_nonnegtive(y,alpha):
    return project_nonnegtive(prox_l1(y,alpha))
prox_l1(-np.array(range(5)),1)


N = 100
c = 3
lambda1 = 1
numworker = 5
maxiter = 500
def generate_component(i):
    if i==0:
        def _fi(x):
            return (x[0]-c)*(x[0]-c)+(x[1]+c)*(x[1]+c)*0.5
        def _nabla_fi(x):
            _tmp = np.zeros(N)
            _tmp[0]=2*x[0]-2*c
            _tmp[1]=x[1]+c
            return _tmp
    elif i==N-1:
        def _fi(x):
            return (x[i-1]+c)*(x[i-1]+c)*0.5+(x[i]-c)*(x[i]-c)*0.5
        def _nabla_fi(x):
            _tmp = np.zeros(N)
            _tmp[i-1]=x[i-1]+c
            _tmp[i]=x[i]-c
            return _tmp
    else:
        def _fi(x):
            return 0.5*(x[i-1]+c)*(x[i-1]+c)+(x[i]-c)*(x[i]-c)*0.5+(x[i+1]+c)*(x[i+1]+c)*0.5
        def _nabla_fi(x):
            _tmp = np.zeros(N)
            _tmp[i-1]=x[i-1]+c
            _tmp[i]=x[i]-c
            _tmp[i+1]=x[i+1]+c
            return _tmp
    return _fi,_nabla_fi

def generate_worker_operator(numworker,nablalist,mode='ordered'):
    N = len(nablalist)
    capicity = N//numworker
    remain = N%numworker
    workerlist = [[] for i in range(numworker)]
    if mode == 'random':
        r=[i for i in range(N)]
        np.random.shuffle(r)
        queuelist = r
    if mode == 'ordered':
        queuelist = range(N)
    i = 0
    for i_worker in range(numworker):
        for i_cap in range(capicity + (1 if i_worker<remain else 0)):
            workerlist[i_worker].append(nablalist[queuelist[i]])
            i+=1
    def _operatorlist_combine(operatorlist):
        def _operatorlist_combined(x):
            res = 0
            for operator in operatorlist:
                res += operator(x)
            return res
        return _operatorlist_combined
    return list(map(_operatorlist_combine,workerlist))

def generate_target_operator(valuelist):
    def _target_function(x):
        s = 0
        for value_function in valuelist:
            s+=value_function(x)
        s = s+lambda1*np.abs(x).sum()
        return s
    return _target_function
        

valuelist,nablalist = zip(*list(map(generate_component,range(N))))

### Prepare the workers
worker_gradient_operator = generate_worker_operator(numworker,nablalist,mode='ordered')
target = generate_target_operator(valuelist)

### Begin the loop
def piag(eta,alpha=0.01):
    ### initialization x
    x = np.ones(N)*10
    xold = np.ones(N)*10
    ### initialization gradient workers
    slave_gradient = np.zeros([numworker,N])
    master_gradient = np.zeros([numworker,N])
    for i in range(numworker):
        slave_gradient[i] = worker_gradient_operator[i](x)
        master_gradient[i] = worker_gradient_operator[i](x)

    resulttable=[]
    for it in range(maxiter):
        i = np.random.randint(numworker)
        ### Update the ith gradient in the master from the ith worker
        master_gradient[i] = slave_gradient[i]
        ### Recalculate the gradient
        gradient = master_gradient.sum(0)
        ### Update x
        y = x - alpha*gradient+eta*(x-xold)
        xold = x.copy()
        x = prox_l1_nonnegtive(y,alpha*lambda1)
        ### Send x to the ith worker
        ### Let the ith worker update the ith gradient
        slave_gradient[i] = worker_gradient_operator[i](x)
        if it%1==0:
            resulttable.append(x)
    return resulttable

### Begin the loop
def piag2(eta,mode="acc3",alpha=0.01,maxiter=500):
    ### initialization x
    x = np.ones(N)
    xold = np.ones(N)
    y = np.ones(N)
    yold = np.ones(N)
    ### initialization gradient workers
    slave_gradient = np.zeros([numworker,N])
    master_gradient = np.zeros([numworker,N])
    for i in range(numworker):
        slave_gradient[i] = worker_gradient_operator[i](x)
        master_gradient[i] = worker_gradient_operator[i](x)

    resulttable=[]
    for it in range(maxiter):
        i = np.random.randint(numworker)
        ### Update the ith gradient in the master from the ith worker
        master_gradient[i] = slave_gradient[i]
        ### Recalculate the gradient
        gradient = master_gradient.sum(0)
        ### Update x
        if mode == "acc1":
            #yold = y.copy()
            y = x - alpha*gradient        
            y = prox_l1_nonnegtive(y,alpha*lambda1)
            x = y + eta*(y-x)
        if mode == "acc2":
            xold = x.copy()
            x = x - alpha*gradient        
            x = prox_l1_nonnegtive(x,alpha*lambda1)
            x = x + eta*(x-xold)
        if mode == "acc3":
            yold = y.copy()       
            y = prox_l1_nonnegtive(x - alpha*gradient,alpha*lambda1)
            x = y+eta*(y-yold)
        if mode == "acc4":
            yold = y.copy()
            y = prox_l1_nonnegtive(y - alpha*gradient,alpha*lambda1)
            x = y+eta*(y-yold)
        if mode == "acc5":
            yold = y.copy()
            y = prox_l1_nonnegtive(x - alpha*gradient+eta*(x-xold),alpha*lambda1)
            xold = x.copy()
            x = y+eta*(y-yold)
        ### Send x to the ith worker
        ### Let the ith worker update the ith gradient
        slave_gradient[i] = worker_gradient_operator[i](x)
        if it%1==0:
            resulttable.append(y)
    return resulttable

def piag3(eta1,eta2,alpha=0.01,maxiter=500):
    ### initialization x
    x = np.ones(N)
    xold = np.ones(N)
    y = np.ones(N)
    yold = np.ones(N)
    z = np.ones(N)
    zold = np.ones(N)
    ### initialization gradient workers
    slave_gradient = np.zeros([numworker,N])
    master_gradient = np.zeros([numworker,N])
    for i in range(numworker):
        slave_gradient[i] = worker_gradient_operator[i](x)
        master_gradient[i] = worker_gradient_operator[i](x)

    resulttable=[]
    for it in range(maxiter):
        i = np.random.randint(numworker)
        ### Update the ith gradient in the master from the ith worker
        master_gradient[i] = slave_gradient[i]
        ### Recalculate the gradient
        gradient = master_gradient.sum(0)
        ### Update x
        
        y = x + eta1*(x-xold)
        zold = z.copy()
        z = prox_l1_nonnegtive(y - alpha*gradient,alpha*lambda1)
        xold = x.copy()
        x = z + 0*(z-zold)

        ### Send x to the ith worker
        ### Let the ith worker update the ith gradient
        slave_gradient[i] = worker_gradient_operator[i](x)
        if it%1==0:
            resulttable.append(y)
    return resulttable



In [ ]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
iters = 5000
x_star = np.zeros(N)
x_star[0] = max(0,(c-lambda1)/3)
F_star = target(x_star).sum(0)
print('F_star is:',F_star )

markerlist = ['s','.','x','*','o']
colorlist = ['royalblue','cornflowerblue','steelblue','skyblue','powderblue']

alpha=0.0066
etatable=[0.0,0.2,0.4,0.6,0.8]

resulttablelistlist = []
Loss_list = np.zeros((5,iters))
for repeat in range(1):
    k = 0
    resulttablelist = []
    for eta in etatable:
        print(eta)
        x = piag2(eta,alpha=alpha,maxiter=iters)
        for i in  range(iters):
            loss_value = target(x[i]).sum(0)
            # print('The '+str(i)+'-th Loss is:', loss_value)
            Loss_list[k,i] = loss_value
        resulttablelist.append(x)
        k = k+1 
    resulttablelistlist.append(resulttablelist)

i=0
resulttablelist = (((np.array(resulttablelistlist)-x_star)**2)).sum(3).mean(0)
plt.figure(num=1)
plt.ylim(1e-30,1e5)
for resulttable in resulttablelist:
    plt.semilogy(range(iters), resulttable, color = colorlist[i], marker=markerlist[i], markevery=200)
    i=i+1

pho=1/(2*alpha+1)
z0=resulttablelistlist[0][0][0]
z1=resulttablelistlist[0][0][1]
thbound=(pho*np.linalg.norm(z1-x_star)**2+np.linalg.norm(z0-x_star)**2+0.5*pho*np.linalg.norm(z1-z0)**2)*pho**np.array(range(iters))
plt.semilogy(thbound, linestyle='dashed',color=[0,0,0])
leg = plt.legend([r'PIAG',r'PIAG with $\beta=0.2$',r'PIAG with $\beta=0.4$',r'PIAG with $\beta=0.6$',r'PIAG with $\beta=0.8$',r'PIAG:Theoretical bound'],loc='upper right')
frame = leg.get_frame()
frame.set_alpha(1)  
frame.set_facecolor('none') 
plt.ylabel(r"$\|x_k-x^\ast\|^2$",
          fontsize=16, color='black')
plt.xlabel(r"Iterate $k$")

plt.savefig("fig/toy_x_alpha_"+str(alpha)+".eps")

plt.figure(num=2)
plt.ylim(1e-15,1e5)
for i in range(5):
    # iter_k = np.array(range(iters))
    # print(iter_k)
    plt.semilogy((Loss_list[i,:]-F_star+1e-32).tolist(), color = colorlist[i], marker=markerlist[i], markevery=200)

pho=1/(2*alpha+1)
# pho = 1/(alpha/8+1)
z0=resulttablelistlist[0][0][0]
z1=resulttablelistlist[0][0][1]
z2=resulttablelistlist[0][0][2]
C1=target(z2).sum(0) - F_star + (target(z1).sum(0) - F_star)* pho + 0.25/alpha *pho * np.linalg.norm(z1-z0)**2
C1 = target(z1).sum(0) - F_star
Loss_thbound=C1*pho**np.array(range(iters))
print(Loss_thbound)
plt.semilogy(Loss_thbound+1e-32, linestyle='dashed',color=[0,0,0])
leg = plt.legend([r'PIAG',r'PIAG with $\beta=0.2$',r'PIAG with $\beta=0.4$',r'PIAG with $\beta=0.6$',r'PIAG with $\beta=0.8$',r'PIAG:Theoretical bound'],loc='upper right')
frame = leg.get_frame()
frame.set_alpha(1)  
frame.set_facecolor('none') 
plt.ylabel(r"$F(x_k)-F^{\ast}$",
          fontsize=16, color='black')
plt.xlabel(r"Iterate $k$")
plt.savefig("fig/toy_loss_alpha_"+str(alpha)+".eps")

plt.show()


In [ ]:

def rho(beta, sigma, L, tau):
 a = 8*(2+beta)/sigma
 Q = L/sigma
 b = (1 + (1-83*beta/16)/(24*(2+beta)*Q))**(1/(1+tau))
 return a * (b-1)

print('r(0):', rho(0,2,101,4))
print('r(0.04):', rho(0.04,2,101,4))
print('r(0.08):', rho(15/83,2,101,4))


r(0): 0.0006599571117700265
r(0.04): 0.0005230352554731966
r(0.08): 4.125373520242919e-05


In [ ]:
from matplotlib import rc
np.random.seed(104)
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
plt.figure(num=1)
x = np.zeros(N)
x[0] = max(0,(c-lambda1)/3)
x = x.reshape((-1,N))

mk = ['o','*','d','s','1','v','p','+','8','h']
i = 0
etatable = [0.0]
# alpha_table= 0.00066 * np.array([2**6,2**6+2**1, 2**6+2**2, 2**6+2**3, 2**6+2**4,  2**6+2**5, 2**7])
# alpha_table= 0.00066 * np.array([2**6, 2**7,2**7+1/2**2,2**7+1/2,2**7+1,2**7+2,2**7+2**2,2**7+2**3,2**7+2**4, 2**7+2**5,2**7+2**6, 2**8,2**9])
alpha_table = 0.00066 * np.array([2**7])
print(alpha_table)
mean_iters = 100
category_colors1 = plt.get_cmap('Blues')(np.linspace(0.1, 0.7, mean_iters))
count_num = np.zeros((len(alpha_table),1))
tol = 1e-25
for alpha in alpha_table:
    for eta in etatable:
        err_total = np.zeros((1000,mean_iters))
        count = 0
        for k in range(mean_iters):
            x_rec = np.array(piag3(eta,0,alpha=alpha,maxiter=1000))
            err = np.sum((x_rec-x)**2,axis=1)+1e-32
            plt.semilogy( err, color=category_colors1[k], alpha=0.3)
            if err[-1] < tol:
                count = count + 1
            err_total[:,k] = err
        count_num[i,0] = count
        print('alpha:', alpha)
        print('beta:', eta)
        print('converged num:', count)
        if eta == 0:
            la = r'PIAG with $\alpha=0.00066 \times 2^7$'
            # la = r'PIAG'
            plt.semilogy(np.mean(err_total,1), color='mediumblue', marker=mk[i], markerfacecolor='none', markeredgecolor='darkblue', markevery=100,  label = la)
        else:
            la = r'iPIAG with $\beta=$' + str(eta)
            plt.semilogy(np.mean(err_total,1), color=[0,0,1], label = la)
        
        i = i+1
print(count_num)

category_colors2 = plt.get_cmap('Oranges')(np.linspace(0.1, 0.5, mean_iters))
etatable=[0.3]
alpha_table = 0.00066 * np.array([2**4])
for alpha in alpha_table:
    for eta in etatable:
        print('alpha:', alpha)
        print('beta:', eta)
        err_total = np.zeros((1000,mean_iters))
        count = 0
        for k in range(mean_iters):
            x_rec = np.array(piag3(eta,0,alpha=alpha,maxiter=1000))
            err = np.sum((x_rec-x)**2,axis=1)+1e-32
            err_total[:,k] = err
            if err[-1] < tol:
                count = count + 1
            plt.semilogy(err, color=category_colors2[k], alpha=0.3)
        print('iPIAG converged num:',count)
        if eta == 0:
            la = r'PIAG with $\alpha=$'+ str(alpha)
            plt.semilogy( np.mean(err_total,1), color=[0,0.1*i,0], alpha= 0.3, label=la)
        else:
            la = r'iPIAG with $\alpha = 0.00066 \times 2^4$, $\beta = 0.3$'
            # la = r'iPIAG with $\beta=$' + str(eta)
            # la = r'iPIAG'
            plt.semilogy( np.mean(err_total,1), color='orangered',marker=mk[i], markerfacecolor='none', markeredgecolor='darkred', markevery=100, label=la)
        
        i = i+1

plt.ylabel(r"$\|x_k-x^\ast\|^2$",fontsize=16)
plt.xlabel(r"Iterate $k$")


plt.legend(bbox_to_anchor=(0.5, 1.1), loc=10, borderaxespad=0, edgecolor='grey')
plt.tight_layout()
plt.savefig('fig/alpha_compare_9.png',dpi=600)
plt.show()

